In [1]:
# Standard packages
import tempfile
import warnings
import urllib
import shutil
import os
# Less standard, but still pip- or conda-installable
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
#import rasterio
import re
import rtree
import shapely
import pickle
import data_eng.az_proc as ap
import data_eng.form_calcs as fc

#from cartopy import crs
import collections
import cv2
import math
from glob import glob
from tqdm.notebook import tqdm_notebook

# Load Data

In [2]:
parent_directory = "//oit-nas-fe13dc.oit.duke.edu//data_commons-borsuk//"

tile_names_tile_urls_complete_array = np.load("image_download_azure/tile_name_tile_url_complete_array.npy")

tiles_labeled = "tile_name_tile_url_labeled.npy"
tiles_labeled_from_complete_set = np.load(tiles_labeled)

tracker_file_path = 'outputs/tile_img_annotation_annotator.npy'
tile_img_annotation = np.load(tracker_file_path)

tiles_errors = 'tile_name_tile_url_error_downloading.npy'
tiles_errors = np.load(tiles_errors)

#create folder to hold tiles in completed dataset
path_to_complete_dataset = "verified/complete_dataset"
path_to_tiles_folder_complete_dataset = os.path.join(parent_directory, path_to_complete_dataset,"tiles")
path_positive_images_complete_dataset = os.path.join(parent_directory, path_to_complete_dataset,"chips_positive")
path_to_verified_sets = os.path.join(parent_directory, path_to_complete_dataset,"verified//verified_sets")

## Tile names and urls for all tiles apart of the dataset (complete array)

In [3]:
tile_names_tile_urls_complete_array = fc.add_formatted_and_standard_tile_names_to_tile_names_time_urls(tile_names_tile_urls_complete_array)
print(tile_names_tile_urls_complete_array.shape)

tile_names_tile_urls_complete_array_unique_standard_tile_names, tile_names_tile_urls_complete_array_unique_formatted_tile_names = fc.unique_formatted_standard_tile_names(tile_names_tile_urls_complete_array)
#Save unique standard tile names
np.save("tile_names_tile_urls_complete_array_unique_standard_tile_names.npy", tile_names_tile_urls_complete_array_unique_standard_tile_names)
#pd.DataFrame(np.load("image_download_azure/tile_name_tile_url_complete_array.npy")).to_csv("image_download_azure/tile_name_tile_url_complete_array.csv")

(2610, 4)
unique formatted tile names (2540, 4)
unique standard tile names (2439, 4)


# Determine the pixel location of the image within the tile

In [ ]:
#unique positive jpgs (file names with the file extension)
unique_positive_jpgs = fc.unique_positive_jpgs_from_parent_directory(parent_directory)

image_characteristics = fc.image_characteristics(path_to_tiles_folder_complete_dataset,unique_positive_jpgs)

image_characteristics.to_csv('image_characteristics.csv')
counterin = 0
counternot = 0 

for unique_jpg in unique_positive_jpgs[:,0]:
    if image_characteristics['six_digit_chip_name'].isin([unique_jpg]).any():
        counterin += 1
    if not image_characteristics['six_digit_chip_name'].isin([unique_jpg]).any():
        counternot += 1
print(counterin, counternot)

In [ ]:
counter_correct_name = 0
counter_incorrect_name = 0
for jpg in jpgs_without_ext:
    jpg_name_in_image_characteristics = [string for string in image_characteristics['six_digit_chip_name'] if jpg in string] #check if a given tile is in the complete_array 
    if len(jpg_name_in_image_characteristics) >= 1:
        counter_correct_name += 1
    else:
        counter_incorrect_name += 1

In [ ]:
#Identify folders containing positive images